In [ ]:
import pandas as pd
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

In [ ]:
correspondance_df = pd.read_csv("ensembl_id_to_gene_symbol_biomart_export.tsv", delimiter="\t", index_col="Gene stable ID")

In [ ]:
# Fix Pcdha10 manually
correspondance_df.loc["ENSMUSG00000007440","Gene name"] = "Pcdha10"

In [ ]:
correspondance_df[correspondance_df.loc[:,"Gene name"].str.startswith("Pcdha")]

In [ ]:
df = pd.read_csv("Yu_First_wave_endocrine_cells.csv")

In [ ]:
df

In [ ]:
df.Symbol.duplicated().sum()

In [ ]:
df = df.rename({"ID": "Ensembl_ID", "Symbol": "gene_id"}, axis=1)
df

In [ ]:
df.loc[:,"tracking_id"] = df.Ensembl_ID#.apply(lambda x: x.split(".")[0])

In [ ]:
# %% There are some duplicated Ensembl ids
#    rows with the highest mean FPKM are kept (as there's no way to distinguish between them otherwise)
def get_largest(rows):
    return rows.loc[rows.mean(axis=1).idxmax()]

df = df.groupby("tracking_id").apply(get_largest).reset_index(drop=True)

In [ ]:
df

In [ ]:
_df = df.merge(correspondance_df, left_on="tracking_id", right_index=True, how="inner").drop_duplicates().dropna()

In [ ]:
_df

In [ ]:
# %% There are some duplicated gene ids
#    rows with the highest mean FPKM are kept (as there's no way to distinguish between them otherwise)
def get_largest(rows):
    return rows.loc[rows.mean(axis=1).idxmax()]

_df = _df.groupby("Gene name").progress_apply(get_largest)

In [ ]:
_df

In [ ]:
_df = _df.drop(["Ensembl_ID", "gene_id", "tracking_id", "Gene name"], axis=1)

In [ ]:
_df.index.name = "gene_id"

In [ ]:
_df[_df.index.str.startswith("Pcdha")]


In [ ]:
_df.index.duplicated().sum()

In [ ]:
_df.to_csv("Yu_First_wave_endocrine_cells.csv", index=True)